In [1]:

import numpy as np
import pandas as pd

def calculate_weights_ahp():
    # Ma trận so sánh cặp
    comparison_matrix = np.array([
        [1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 9.0],
        [1/3, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
        [1/4, 1/3, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0],
        [1/5, 1/4, 1/3, 1.0, 3.0, 4.0, 5.0, 6.0, 7.0],
        [1/6, 1/5, 1/4, 1/3, 1.0, 3.0, 4.0, 5.0, 6.0],
        [1/7, 1/6, 1/5, 1/4, 1/3, 1.0, 3.0, 4.0, 5.0],
        [1/8, 1/7, 1/6, 1/5, 1/4, 1/3, 1.0, 3.0, 4.0],
        [1/9, 1/8, 1/7, 1/6, 1/5, 1/4, 1/3, 1.0, 3.0],
        [1/9, 1/9, 1/8, 1/7, 1/6, 1/5, 1/4, 1/3, 1.0]
    ])

    
    # Số lượng tiêu chí (n)
    n = comparison_matrix.shape[0]

    # Chuẩn hóa ma trận bằng cách chia từng phần tử cho tổng cột
    column_sums = np.sum(comparison_matrix, axis=0)
    normalized_matrix = comparison_matrix / column_sums

    # Tính vector trọng số (trung bình hàng của ma trận đã chuẩn hóa)
    weights = np.mean(normalized_matrix, axis=1)

    # Tính λ_max theo công thức trong ảnh
    weighted_sum_vector = np.dot(comparison_matrix, weights)
    lambda_max = np.sum(weighted_sum_vector / weights) / n

    # Tính chỉ số nhất quán CI
    CI = (lambda_max - n) / (n - 1)

    # Giá trị RI (Random Index) tra bảng
    RI = {1: 0, 2: 0, 3: 0.58, 4: 0.9, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45}
    RI_value = RI.get(n, 1)  # Tra giá trị RI theo n

    # Tính chỉ số nhất quán CR
    CR = CI / RI_value

    return weights, lambda_max, CI, CR

In [11]:
def classify_landslide_risk(df, weights):
    columns = ['slope', 'elevation', 'thachhoc', 'dis_dutgay', 'dis_thachhoc', 
               'dis_river', 'dis_mainroad', 'dis_road', 'dis_dancu']
    
    # Chuẩn hóa các cột chỉ một lần bằng cách vector hóa
    min_vals = df[columns].min()
    max_vals = df[columns].max()
    normalized_df = (df[columns] - min_vals) / (max_vals - min_vals)
    
    # Tính LSI bằng cách nhân ma trận thay vì vòng lặp
    lsi = normalized_df.dot(weights)
    
    # Tính ngưỡng và phân loại dựa trên vector hóa
    min_lsi, max_lsi = lsi.min(), lsi.max()
    interval = (max_lsi - min_lsi) / 5  # Chia thành 5 khoảng bằng nhau
    thresholds = [min_lsi + i * interval for i in range(1, 5)]

    print(thresholds)
    
    # Hàm phân loại vector hóa
    risk_levels = pd.cut(
        lsi, 
        bins=[-np.inf] + thresholds + [np.inf],
        labels=['Rất thấp', 'Thấp', 'Trung bình', 'Cao', 'Rất cao']
    )
    
    return risk_levels, lsi



In [3]:

# %%
def print_lsi_statistics(df1, df2):
    """
    In thống kê chi tiết về phân bố LSI cho cả khu vực có trượt và không có trượt
    theo thứ tự các mức độ đã định nghĩa
    """
    # Kết hợp tất cả dữ liệu
    df1['Loai'] = 'Có trượt'
    df2['Loai'] = 'Không trượt'
    combined_df = pd.concat([df1, df2])
    
    # Định nghĩa thứ tự các mức độ
    risk_order = ['Rất thấp', 'Thấp', 'Trung bình', 'Cao', 'Rất cao']
    
    # Tính toán thống kê tổng thể
    total_stats = combined_df.groupby('Nguy_co').agg({
        'LSI': ['count', 'min', 'max', 'mean'],
        'Loai': lambda x: x.value_counts().to_dict()
    }).round(4)
    
    # Tính tỷ lệ phần trăm
    total_count = len(combined_df)
    total_stats[('LSI', 'percentage')] = (total_stats[('LSI', 'count')] / total_count * 100).round(2)
    
    # Sắp xếp lại theo thứ tự định nghĩa
    total_stats = total_stats.reindex(risk_order)
    
    # Định dạng lại kết quả
    print("\n=== THỐNG KÊ CHI TIẾT PHÂN VÙNG NGUY CƠ TRƯỢT LỞ ===\n")
    print("1. Thống kê tổng thể:")
    print("-" * 100)
    print(f"{'Mức độ':<15} {'Số lượng':<10} {'Tỷ lệ %':<10} {'LSI Min':<10} {'LSI Max':<10} {'LSI TB':<10}")
    print("-" * 100)
    
    for idx in risk_order:
        if idx in total_stats.index:
            count = total_stats.loc[idx, ('LSI', 'count')]
            percentage = total_stats.loc[idx, ('LSI', 'percentage')]
            lsi_min = total_stats.loc[idx, ('LSI', 'min')]
            lsi_max = total_stats.loc[idx, ('LSI', 'max')]
            lsi_mean = total_stats.loc[idx, ('LSI', 'mean')]
            type_counts = total_stats.loc[idx, ('Loai', '<lambda>')]
           
            
            print(f"{idx:<15} {count:<10} {percentage:<10} {lsi_min:<10.4f} {lsi_max:<10.4f} "
                  f"{lsi_mean:<10.4f}")
    
    print("-" * 100)
    print(f"Tổng số mẫu: {total_count}")

    print("\n2. Thống kê theo khu vực:")
    print("\nKhu vực có trượt:")
    print("-" * 60)
    print(f"{'Mức độ':<15} {'Số lượng':<10} {'Tỷ lệ %':<10} {'LSI Min':<10} {'LSI Max':<10}")
    print("-" * 60)
    
    for idx in risk_order:
        subset = df1[df1['Nguy_co'] == idx]
        if len(subset) > 0:
            count = len(subset)
            percentage = round((count / len(df1) * 100), 2)  # Sửa ở đây
            lsi_stats = subset['LSI'].agg(['min', 'max', 'mean']).round(4)
            print(f"{idx:<15} {count:<10} {percentage:<10} {lsi_stats['min']:<10.4f} "
                  f"{lsi_stats['max']:<10.4f}")
    
    print("-" * 60)
    print(f"Tổng số mẫu có trượt: {len(df1)}")
    
    print("\nKhu vực không có trượt:")
    print("-" * 60)
    print(f"{'Mức độ':<15} {'Số lượng':<10} {'Tỷ lệ %':<10} {'LSI Min':<10} {'LSI Max':<10} {'LSI TB'}")
    print("-" * 60)
    
    for idx in risk_order:
        subset = df2[df2['Nguy_co'] == idx]
        if len(subset) > 0:
            count = len(subset)
            percentage = round((count / len(df2) * 100), 2)  
            lsi_stats = subset['LSI'].agg(['min', 'max', 'mean']).round(4)
            print(f"{idx:<15} {count:<10} {percentage:<10} {lsi_stats['min']:<10.4f} "
                  f"{lsi_stats['max']:<10.4f} {lsi_stats['mean']:<10.4f}")
    
    print("-" * 60)
    print(f"Tổng số mẫu không có trượt: {len(df2)}")




In [4]:

df1 = pd.read_csv('data/CSDL_YenBai_CoTruot.csv')
df2 = pd.read_csv('data/CSDL_YenBai_KoTruot.csv')


weights, lambda_max, ci, cr = calculate_weights_ahp()
print(f"CI: {ci:.4f}")
print(f"Consistency Ratio: {cr:.4f}")
if cr > 0.1:
    print("Cảnh báo: CR > 0.1, cần xem xét lại ma trận so sánh cặp")


criteria = ['Độ dốc', 'Độ cao', 'Thạch học', 'Khoảng cách đứt gãy', 
            'Khoảng cách thạch học', 'Khoảng cách sông suối', 
            'Khoảng cách đường chính', 'Khoảng cách đường', 'Khoảng cách dân cư']
print("\nTrọng số các tiêu chí:")
for c, w in zip(criteria, weights):
    print(f"{c}: {w:.4f}")


risk_levels1, lsi1 = classify_landslide_risk(df1, weights)
risk_levels2, lsi2 = classify_landslide_risk(df2, weights)


df1['Nguy_co'] = risk_levels1
df1['LSI'] = lsi1
df2['Nguy_co'] = risk_levels2
df2['LSI'] = lsi2


print_lsi_statistics(df1, df2)



CI: 0.1434
Consistency Ratio: 0.0989

Trọng số các tiêu chí:
Độ dốc: 0.3262
Độ cao: 0.2187
Thạch học: 0.1514
Khoảng cách đứt gãy: 0.1059
Khoảng cách thạch học: 0.0739
Khoảng cách sông suối: 0.0509
Khoảng cách đường chính: 0.0345
Khoảng cách đường: 0.0229
Khoảng cách dân cư: 0.0156

=== THỐNG KÊ CHI TIẾT PHÂN VÙNG NGUY CƠ TRƯỢT LỞ ===

1. Thống kê tổng thể:
----------------------------------------------------------------------------------------------------
Mức độ          Số lượng   Tỷ lệ %    LSI Min    LSI Max    LSI TB    
----------------------------------------------------------------------------------------------------
Rất thấp        3563       1.47       0.0879     0.3732     0.2060    
Thấp            29704      12.23      0.2525     0.5064     0.3577    
Trung bình      82059      33.78      0.4171     0.6398     0.5147    
Cao             118790     48.9       0.5817     0.7747     0.6490    
Rất cao         8790       3.62       0.7463     0.9108     0.7817    
-------------

In [5]:
print(df1.columns)
print(df2.columns)

Index(['pointid', 'ma_cu', 'ma_moi', 'slope', 'elevation', 'thachhoc',
       'dis_dutgay', 'dis_thachhoc', 'dis_river', 'dis_mainroad', 'dis_road',
       'dis_dancu', 'output', 'Nguy_co', 'LSI', 'Loai'],
      dtype='object')
Index(['pointid', 'ma_cu', 'ma_moi', 'slope', 'elevation', 'thachhoc',
       'dis_dutgay', 'dis_thachhoc', 'dis_river', 'dis_mainroad', 'dis_road',
       'dis_dancu', 'output', 'Nguy_co', 'LSI', 'Loai'],
      dtype='object')


In [6]:
weights

array([0.32619072, 0.21869962, 0.15138098, 0.10593059, 0.0739084 ,
       0.05094576, 0.03448381, 0.0228982 , 0.01556192])

In [16]:

classify_landslide_risk(df2, weights)[0]

[0.2524882213054687, 0.41707648652305984, 0.5816647517406509, 0.746253016958242]


0                Cao
1                Cao
2                Cao
3                Cao
4                Cao
             ...    
240812           Cao
240813           Cao
240814    Trung bình
240815    Trung bình
240816    Trung bình
Length: 240817, dtype: category
Categories (5, object): ['Rất thấp' < 'Thấp' < 'Trung bình' < 'Cao' < 'Rất cao']

In [19]:
df1

,pointid,ma_cu,ma_moi,slope,elevation,thachhoc,dis_dutgay,dis_thachhoc,dis_river,dis_mainroad,dis_road,dis_dancu,output,Nguy_co,LSI,Loai
0,1088835,1088835,1088835,5,4,4,3,4,1,4,0,3,1,Rất cao,0.908519,Có trượt
1,1088840,1088840,1088840,5,4,4,3,4,1,4,0,2,1,Rất cao,0.904629,Có trượt
2,1138568,1138568,1138568,2,4,4,0,0,1,0,2,0,1,Trung bình,0.524742,Có trượt
3,1138579,1138579,1138579,2,4,4,0,0,1,0,2,0,1,Trung bình,0.524742,Có trượt
4,1138595,1138595,1138595,2,4,4,0,0,1,0,2,0,1,Trung bình,0.524742,Có trượt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,1187528,1187528,1187528,4,4,4,0,0,2,0,3,1,1,Cao,0.677570,Có trượt
2085,1182827,1182827,1182827,4,4,4,1,0,0,0,4,4,1,Cao,0.695976,Có trượt
2086,1254090,1254090,1254090,4,3,3,3,4,3,0,4,2,1,Cao,0.722913,Có trượt
2087,1254091,1254091,1254091,4,3,3,3,4,3,0,4,2,1,Cao,0.722913,Có trượt
